In [ ]:
# Import packages
from dash import Dash, html, dash_table
import pandas as pd
# final_dfs_dict
df = pd.read_csv('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-Epoch.csv')

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My First App with Data'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10)
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc # (DCC stands for Dash Core Components)
# dcc.Graph: used to render interactive graphs.
import pandas as pd
import plotly.io as pio
template: str = 'plotly_dark' # set plotl template
pio.templates.default = template
import plotly.express as px

# Incorporate data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My First App with Data and a Graph'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=10),
    dcc.Graph(figure=px.histogram(df, x='continent', y='lifeExp', histfunc='avg'))
])

# Run the app
if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# Incorporate data
df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder2007.csv')

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My First App with Data, Graph, and Controls'),
    html.Hr(),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph'),
    dcc.RadioItems(options=['pop', 'lifeExp', 'gdpPercap'], value='lifeExp', id='controls-and-radio-item'),
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
from pathlib import Path
import plotly.express as px

# Incorporate data
final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-TimeBin.csv')}

final_dfs_dict = {a_name:pd.read_csv(a_path.resolve()) for a_name, a_path in final_csv_export_paths.items()}

options_list = list(final_csv_export_paths.keys())
initial_option = options_list[0]

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My Custom App with Data, Graph, and Controls'),
    html.Hr(),
    dcc.RadioItems(options=options_list, value=initial_option, id='controls-and-radio-item'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph'),
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    print(f'col_chosen: {col_chosen}')
    a_df = final_dfs_dict[col_chosen]
    # fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    # fig = px.histogram(a_df, x='continent', y='P_Long', histfunc='avg')
    fig = px.scatter(a_df, x='delta_aligned_start_t', y='P_Long', title=f"{col_chosen}", color='session_name', size='time_bin_size')
    return fig


    # fig = px.histogram(df, x='continent', y=col_chosen, histfunc='avg')
    # return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [2]:
from dash import Dash, html, dcc, Input, Output, callback
import pandas as pd
import plotly.express as px

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = Dash(__name__, external_stylesheets=external_stylesheets)

df = pd.read_csv('https://plotly.github.io/datasets/country_indicators.csv')


app.layout = html.Div([
    html.Div([

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Fertility rate, total (births per woman)',
                id='crossfilter-xaxis-column',
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-xaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ],
        style={'width': '49%', 'display': 'inline-block'}),

        html.Div([
            dcc.Dropdown(
                df['Indicator Name'].unique(),
                'Life expectancy at birth, total (years)',
                id='crossfilter-yaxis-column'
            ),
            dcc.RadioItems(
                ['Linear', 'Log'],
                'Linear',
                id='crossfilter-yaxis-type',
                labelStyle={'display': 'inline-block', 'marginTop': '5px'}
            )
        ], style={'width': '49%', 'float': 'right', 'display': 'inline-block'})
    ], style={
        'padding': '10px 5px'
    }),

    html.Div([
        dcc.Graph(
            id='crossfilter-indicator-scatter',
            hoverData={'points': [{'customdata': 'Japan'}]}
        )
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
    html.Div([
        dcc.Graph(id='x-time-series'),
        dcc.Graph(id='y-time-series'),
    ], style={'display': 'inline-block', 'width': '49%'}),

    html.Div(dcc.Slider(
        df['Year'].min(),
        df['Year'].max(),
        step=None,
        id='crossfilter-year--slider',
        value=df['Year'].max(),
        marks={str(year): str(year) for year in df['Year'].unique()}
    ), style={'width': '49%', 'padding': '0px 20px 20px 20px'})
])


@callback(
    Output('crossfilter-indicator-scatter', 'figure'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'),
    Input('crossfilter-yaxis-type', 'value'),
    Input('crossfilter-year--slider', 'value'))
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type,
                 year_value):
    dff = df[df['Year'] == year_value]

    fig = px.scatter(x=dff[dff['Indicator Name'] == xaxis_column_name]['Value'],
            y=dff[dff['Indicator Name'] == yaxis_column_name]['Value'],
            hover_name=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name']
            )

    fig.update_traces(customdata=dff[dff['Indicator Name'] == yaxis_column_name]['Country Name'])

    fig.update_xaxes(title=xaxis_column_name, type='linear' if xaxis_type == 'Linear' else 'log')

    fig.update_yaxes(title=yaxis_column_name, type='linear' if yaxis_type == 'Linear' else 'log')

    fig.update_layout(margin={'l': 40, 'b': 40, 't': 10, 'r': 0}, hovermode='closest')

    return fig


def create_time_series(dff, axis_type, title):

    fig = px.scatter(dff, x='Year', y='Value')

    fig.update_traces(mode='lines+markers')

    fig.update_xaxes(showgrid=False)

    fig.update_yaxes(type='linear' if axis_type == 'Linear' else 'log')

    fig.add_annotation(x=0, y=0.85, xanchor='left', yanchor='bottom',
                       xref='paper', yref='paper', showarrow=False, align='left',
                       text=title)

    fig.update_layout(height=225, margin={'l': 20, 'b': 30, 'r': 10, 't': 10})

    return fig


@callback(
    Output('x-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-xaxis-column', 'value'),
    Input('crossfilter-xaxis-type', 'value'))
def update_x_timeseries(hoverData, xaxis_column_name, axis_type):
    country_name = hoverData['points'][0]['customdata']
    dff = df[df['Country Name'] == country_name]
    dff = dff[dff['Indicator Name'] == xaxis_column_name]
    title = '<b>{}</b><br>{}'.format(country_name, xaxis_column_name)
    return create_time_series(dff, axis_type, title)


@callback(
    Output('y-time-series', 'figure'),
    Input('crossfilter-indicator-scatter', 'hoverData'),
    Input('crossfilter-yaxis-column', 'value'),
    Input('crossfilter-yaxis-type', 'value'))
def update_y_timeseries(hoverData, yaxis_column_name, axis_type):
    dff = df[df['Country Name'] == hoverData['points'][0]['customdata']]
    dff = dff[dff['Indicator Name'] == yaxis_column_name]
    return create_time_series(dff, axis_type, yaxis_column_name)


if __name__ == '__main__':
    app.run(debug=True)


# Four-CSV displaying version

In [ ]:
# Import packages
from typing import List, Dict, Optional, Tuple, Union
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
from pathlib import Path
import plotly.express as px

# Incorporate data
final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-TimeBin.csv')}

final_dfs_dict = {a_name:pd.read_csv(a_path.resolve()) for a_name, a_path in final_csv_export_paths.items()}

options_list = list(final_csv_export_paths.keys())
initial_option = options_list[0]
initial_dataframe: pd.DataFrame = final_dfs_dict[initial_option]

# Initialize the app
app = Dash(__name__)

# App layout
app.layout = html.Div([
    html.Div(children='My Custom App with Data, Graph, and Controls'),
    html.Hr(),
    dcc.RadioItems(options=options_list, value=initial_option, id='controls-and-radio-item'),
    dash_table.DataTable(data=initial_dataframe.to_dict('records'), page_size=6, id='tbl-datatable'),
    dcc.Graph(figure={}, id='controls-and-graph'),
])

# Add controls to build the interaction
@callback(
    Output(component_id='controls-and-graph', component_property='figure'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_graph(col_chosen):
    print(f'update_graph(col_chosen: {col_chosen})')
    data_results_df: pd.DataFrame = final_dfs_dict[col_chosen]
    unique_sessions: List[str] = data_results_df['session_name'].unique().tolist()
    num_unique_sessions: int = data_results_df['session_name'].nunique(dropna=True) # number of unique sessions, ignoring the NA entries

    ## Extract the unique time bin sizes:
    time_bin_sizes: List[float] = data_results_df['time_bin_size'].unique().tolist()
    num_unique_time_bins: int = data_results_df.time_bin_size.nunique(dropna=True)
    print(f'num_unique_sessions: {num_unique_sessions}, num_unique_time_bins: {num_unique_time_bins}')
    fig = px.scatter(data_results_df, x='delta_aligned_start_t', y='P_Long', title=f"{col_chosen}", color='time_bin_size', range_y=[0.0, 1.0],
                            labels={"session_name": "Session", "time_bin_size": "tbin_size"},
                            facet_row='session_name', facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
                            height=num_unique_sessions*200, width=1024,
                            )
    return fig


@callback(
    Output(component_id='tbl-datatable', component_property='data'),
    Input(component_id='controls-and-radio-item', component_property='value')
)
def update_datatable(col_chosen):
    print(f'update_datatable(col_chosen: {col_chosen})')
    a_df = final_dfs_dict[col_chosen]
    data = a_df.to_dict('records')
    return data


# Run the app
if __name__ == '__main__':
    app.run(debug=True)

In [1]:
# Import packages
from typing import List, Dict, Optional, Tuple, Union
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State
import dash_bootstrap_components as dbc
import pandas as pd
from pathlib import Path
import plotly.express as px


def _build_dash_app(final_dfs_dict):
    ## DATA:    
    options_list = list(final_dfs_dict.keys())
    initial_option = options_list[0]
    initial_dataframe: pd.DataFrame = final_dfs_dict[initial_option]
    unique_sessions: List[str] = initial_dataframe['session_name'].unique().tolist()
    num_unique_sessions: int = initial_dataframe['session_name'].nunique(dropna=True) # number of unique sessions, ignoring the NA entries

    ## Extract the unique time bin sizes:
    time_bin_sizes: List[float] = initial_dataframe['time_bin_size'].unique().tolist()
    num_unique_time_bins: int = initial_dataframe.time_bin_size.nunique(dropna=True)
    print(f'num_unique_sessions: {num_unique_sessions}, num_unique_time_bins: {num_unique_time_bins}')
    enabled_time_bin_sizes = [time_bin_sizes[0], time_bin_sizes[-1]] # [0.03, 0.058, 0.10]

    # Initialize the app
    app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

    # App layout
    app.layout = dbc.Container([
        html.Div(children='My Custom App with Data, Graph, and Controls'),
        html.Hr(),
        dbc.Row([
            dbc.Col(dcc.RadioItems(options=options_list, value=initial_option, id='controls-and-radio-item')),
            # Add CheckboxGroup for time_bin_sizes
            # html.Div(dcc.Checklist(options=[{'label':i, 'value': i} for i in final_dfs_dict[initial_option]['time_bin_size'].unique()], id='time-bin-checkboxes', inline=True)),
            dbc.Col(dcc.Checklist(options=time_bin_sizes, value=enabled_time_bin_sizes, id='time-bin-checkboxes', inline=True)),
        ]),
        dbc.Row(dash_table.DataTable(data=initial_dataframe.to_dict('records'), page_size=6, id='tbl-datatable')),
        dbc.Row(dcc.Graph(figure={}, id='controls-and-graph')),
    ])


    # Add controls to build the interaction
    @callback(
        Output(component_id='controls-and-graph', component_property='figure'),
        [Input(component_id='controls-and-radio-item', component_property='value'),
            State(component_id='time-bin-checkboxes', component_property='value'),
        ]
    )
    def update_graph(col_chosen, chose_bin_sizes):
        print(f'update_graph(col_chosen: {col_chosen}, chose_bin_sizes: {chose_bin_sizes})')
        data_results_df: pd.DataFrame = final_dfs_dict[col_chosen]
        # Filter dataframe by chosen bin sizes
        data_results_df = data_results_df[data_results_df.time_bin_size.isin(chose_bin_sizes)]
        
        unique_sessions: List[str] = data_results_df['session_name'].unique().tolist()
        num_unique_sessions: int = data_results_df['session_name'].nunique(dropna=True) # number of unique sessions, ignoring the NA entries

        ## Extract the unique time bin sizes:
        time_bin_sizes: List[float] = data_results_df['time_bin_size'].unique().tolist()
        num_unique_time_bins: int = data_results_df.time_bin_size.nunique(dropna=True)
        print(f'num_unique_sessions: {num_unique_sessions}, num_unique_time_bins: {num_unique_time_bins}')
        fig = px.scatter(data_results_df, x='delta_aligned_start_t', y='P_Long', title=f"{col_chosen}", color='time_bin_size', range_y=[0.0, 1.0],
                                labels={"session_name": "Session", "time_bin_size": "tbin_size"},
                                facet_row='session_name', facet_row_spacing=0.04, # default is 0.07 when facet_col_wrap is used
                                facet_col_wrap=2, facet_col_spacing=0.04,
                                height=num_unique_sessions*200, width=1024,
                                )
        return fig


    @callback(
        Output(component_id='tbl-datatable', component_property='data'),
        [Input(component_id='controls-and-radio-item', component_property='value'),
            State(component_id='time-bin-checkboxes', component_property='value'),
        ]
    )
    def update_datatable(col_chosen, chose_bin_sizes):
        print(f'update_datatable(col_chosen: {col_chosen}, chose_bin_sizes: {chose_bin_sizes})')
        a_df = final_dfs_dict[col_chosen]
        # Filter dataframe by chosen bin sizes
        a_df = a_df[a_df.time_bin_size.isin(chose_bin_sizes)]
        data = a_df.to_dict('records')
        return data

    return app


# Incorporate data
final_csv_export_paths = {'AcrossSession_Laps_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-Epoch.csv'),
 'AcrossSession_Ripple_per-Epoch': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-Epoch.csv'),
 'AcrossSession_Laps_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Laps_per-TimeBin.csv'),
 'AcrossSession_Ripple_per-TimeBin': Path('C:/Users/pho/repos/Spike3DWorkEnv/Spike3D/output/2024-01-29_AcrossSession_Ripple_per-TimeBin.csv')}

final_dfs_dict = {a_name:pd.read_csv(a_path.resolve()) for a_name, a_path in final_csv_export_paths.items()}
app = _build_dash_app(final_dfs_dict)

# Run the app
if __name__ == '__main__':
    app.run(port="8052", debug=True)

num_unique_sessions: 13, num_unique_time_bins: 6


In [ ]:
import plotly.graph_objects as go

import numpy as np

y = np.random.randn(500)
# Use `y` argument instead of `x` for horizontal histogram

fig = go.Figure(data=[go.Histogram(y=y)])
fig.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go
import numpy as np

# assuming X and Y are your data
# also assuming that the scatter is sorted in the way you want to split it
X = np.random.randn(200)
Y = np.random.randn(200)

split = len(X) // 2  # get the point to split the data

# creating subplots
fig = sp.make_subplots(rows=1, cols=3, column_widths=[0.10, 0.80, 0.10], horizontal_spacing=0.01)

# adding first histogram
fig.add_trace(
    go.Histogram(
        y=X[:split],
        name='first half',
        marker_color='#EB89B5'
    ),
    row=1, col=1
)

# adding scatter plot
fig.add_trace(
    go.Scatter(
        x=X,
        y=Y,
        mode='markers',
        marker_color='rgba(152, 0, 0, .8)',
    ),
    row=1, col=2
)

# adding the second histogram
fig.add_trace(
    go.Histogram(
        y=X[split:],
        name='second half',
        marker_color='#330C73'
    ),
    row=1, col=3
)

# Plot!
fig.show()

In [ ]:
import plotly.subplots as sp
import plotly.graph_objs as go
import numpy as np

# assuming X and Y are your data
# also assuming that the scatter is sorted in the way you want to split it
X = np.random.randn(200)
Y = np.random.randn(200)

split = len(X) // 2  # get the point to split the data

# creating subplots
fig = sp.make_subplots(rows=1, cols=3, column_widths=[0.10, 0.80, 0.10], horizontal_spacing=0.01)

# adding first histogram
# Calculate the histogram data
hist1, bins1 = np.histogram(X[:split], bins='auto')

# Adding the first histogram as a bar graph and making x negative
fig.add_trace(
    go.Bar(
        x=-bins1[:-1],
        y=hist1,
        marker_color='#EB89B5',
        name='first half',
		orientation='h',
    ),
    row=1, col=1
)


# adding scatter plot
fig.add_trace(
    go.Scatter(
        x=X,
        y=Y,
        mode='markers',
        marker_color='rgba(152, 0, 0, .8)',
    ),
    row=1, col=2
)

# adding the second histogram

# Calculate the histogram data for second half
hist2, bins2 = np.histogram(X[split:], bins='auto')

# Adding the second histogram
fig.add_trace(
    go.Bar(
        x=bins2[:-1],
        y=hist2,
        marker_color='#330C73',
        name='second half',
		orientation='h',
    ),
    row=1, col=3
)


# Plot!
fig.show()








# Templates

In [ ]:
import plotly.io as pio
pio.templates

template: str = 'plotly_dark'
template=template, 